In [1]:
import torch 
import json,pickle,math
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
LSM = pickle.load(open('./davis_ligand_similarity_matrix.pkl', 'rb'))
PSM = pickle.load(open('./davis_protein_similarity_matrix.pkl', 'rb'))
full_df = pd.read_csv(open('./davis_all_pairs.csv','r'))

In [3]:
# df

In [4]:
SMILES = json.load(open('./data/DAVIS/SMILES.txt'))
TARGETS = json.load(open('./data/DAVIS/target_seq.txt'))
SMILES=list(SMILES.values())
TARGETS=list(TARGETS.values())

In [5]:
all_9_folds={}
for i in [0,1,2]:
    for j in [0,1,2]:
        file_name = 'fold' +str(i) +str(j) 
        
        temp = open('./data/davis/DAVIS_9_FOLDS/' + file_name +'.pkl', 'rb')
        new_df = pd.read_pickle(temp)
        all_9_folds.update({file_name:new_df})
        temp.close()
        

In [6]:
def create_davis_test_train(test_fold_number,all_9_folds):
    test_protein_fold_id = test_fold_number[0]
    test_ligand_fold_id = test_fold_number[1]
    test_set = pd.DataFrame(columns = full_df.columns)
    train_set = pd.DataFrame(columns= full_df.columns)
    for i in [0,1,2]:
        for j in [0,1,2]:
            fold_name = 'fold' + str(i) + str(j)
            df = all_9_folds[fold_name]
            
            if str(i) == test_protein_fold_id and str(j) == test_ligand_fold_id:
                test_set = df.copy()
                
            if str(i) != test_protein_fold_id and str(j) != test_ligand_fold_id:
                train_set = pd.concat([train_set, df.copy()], ignore_index=True)
                
                
    return train_set, test_set


# Create train test split on these 9 folds
## fold_number is the id of fold. For example, test = fold00, train = fold 11,22,12,21

In [7]:
fold_number = '11'

In [8]:
train, test = create_davis_test_train(test_fold_number=fold_number, all_9_folds=all_9_folds)

# To ensure that there are no common targets or drugs in train and test


In [9]:
test_smiles = list(test['SMILES'])
test_targets = list(test['Target Sequence'])
train_smiles = list(train['SMILES'])
train_targets = list(train['Target Sequence'])

for i in test_smiles:
    if i in train_smiles:
        print("common entity present")
for i in test_targets:
    if i in train_targets:
        print("common entity present")


# Creating outer products for test set

In [10]:
outer_test_prods = []
for i,row in test.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_test_prods.append([ki_x_kj])
outer_test_prods = np.array(outer_test_prods)
print(np.shape(outer_test_prods))

(3381, 1, 68, 442)


# creating outer products for train set

In [11]:
outer_train_prods = []
for i,row in train.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_train_prods.append([ki_x_kj])
outer_train_prods = np.array(outer_train_prods)
print(np.shape(outer_train_prods))

(13275, 1, 68, 442)


In [12]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
# num_classes = 10
batch_size = 32
learning_rate = 0.001

In [13]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, outer_prods, transform=None):
#         self.df = pd.read_csv(open(csv_file))
        self.df = dataframe
#         self.root_dir = root_dir
        self.transform = transform
        self.outer_prods = outer_prods
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        output = {'outer_product': self.outer_prods[idx] , 'Label':self.df.iloc[idx]['Label']}
        return output

In [14]:
train_dataset = custom_dataset(dataframe = train, outer_prods = outer_train_prods)
test_dataset = custom_dataset(dataframe = test, outer_prods = outer_test_prods)


In [15]:
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [16]:
print(len(train_loader)*32, len(test_loader)*32)

13280 3392


In [17]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32, 5).double()
        self.pool1 = nn.MaxPool2d(2,2).double()
        self.conv2 = nn.Conv2d(32,18,3).double()
        self.pool2 = nn.MaxPool2d(2,2).double()
        self.fc1 = nn.Linear(18*15*108, 128).double()
        self.fc2 = nn.Linear(128,1).double()
        self.dropout = nn.Dropout(0.1).double()
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,18*15*108)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        
        return x
    

In [18]:
# for i in test_loader:
#     a = i['outer_product']
#     b= i['Label']
#     break
# conv1 = nn.Conv2d(1,32,5).double()
# pool = nn.MaxPool2d(2,2).double()
# conv2 = nn.Conv2d(32,18,3).double()
# fc1 = nn.Linear(18*15*108, 128).double()
# fc2 = nn.Linear(128,1).double()
# dropout = nn.Dropout(0.1).double()
# x= conv1(a)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x= conv2(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x = x.view(-1,18*15*108)
# print(x.shape)
# x = dropout(x)
# print(x.shape)
# x = fc1(x)
# print(x.shape)
# x = fc2(x)
# print(x.shape)

In [19]:
model = ConvNet().to(device)

In [20]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Evaluation metrics

In [21]:
def rmse(y,f):
    rmse = math.sqrt(((y - f)**2).mean(axis=0))
    return rmse
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse
def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp
from lifelines.utils import concordance_index
def ci(y,f):
    return concordance_index(y,f)

In [22]:
def predicting(model, device, test_loader):
    model.eval()
    total_preds = np.array([])
    total_labels = np.array([])
    with torch.no_grad():
        for i in test_loader:
            images = i['outer_product']
            labels = i['Label']
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images) 
            outputs = outputs.cpu().detach().numpy().flatten()
            labels =labels.cpu().detach().numpy().flatten()
            total_preds = np.concatenate([total_preds, outputs])
            total_labels = np.concatenate([total_labels, labels])
    
    model.train()
    return total_labels, total_preds

# Train the model

In [23]:
model_file_name = './SAVED_MODELS/best_sim-CNN-DTA_davis_fold' + fold_number +  '.model'
result_file_name = './SAVED_MODELS/best_result_sim-CNNDTA_davis_fold'+fold_number + '.csv'

In [24]:
# Train the model
best_mse = 1000
best_ci = 0

total_step = len(train_loader)
for epoch in range(num_epochs):
    c=0
    for i in train_loader:
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.flatten(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, c, total_step, loss.item()))
    
    # taking best model so far
    G,P = predicting(model, device, test_loader)
    ret = [rmse(G, P), mse(G, P), pearson(G, P), ci(G, P)]
    if ret[1] < best_mse:
        torch.save(model.state_dict(), model_file_name)
        with open(result_file_name, 'w') as f:
            f.write(','.join(map(str, ret)))
        best_epoch = epoch+1
        best_mse = ret[1]
        best_ci = ret[-1]
        best_r = ret[2]
        
        print('rmse improved at epoch ', best_epoch,
                      '; best_mse,best_ci,best_r:', best_mse, best_ci,best_r)
        
        

/home2/kanakala.ganesh/miniconda3/envs/geometric/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/20], Step [1/415], Loss: 34.8231
Epoch [1/20], Step [2/415], Loss: 14.1588
Epoch [1/20], Step [3/415], Loss: 0.9920
Epoch [1/20], Step [4/415], Loss: 11.0224
Epoch [1/20], Step [5/415], Loss: 6.9615
Epoch [1/20], Step [6/415], Loss: 1.2091
Epoch [1/20], Step [7/415], Loss: 0.7153
Epoch [1/20], Step [8/415], Loss: 2.2669
Epoch [1/20], Step [9/415], Loss: 3.1779
Epoch [1/20], Step [10/415], Loss: 2.5963
Epoch [1/20], Step [11/415], Loss: 2.0872
Epoch [1/20], Step [12/415], Loss: 1.4502
Epoch [1/20], Step [13/415], Loss: 0.2799
Epoch [1/20], Step [14/415], Loss: 0.4762
Epoch [1/20], Step [15/415], Loss: 1.0901
Epoch [1/20], Step [16/415], Loss: 1.6096
Epoch [1/20], Step [17/415], Loss: 1.5560
Epoch [1/20], Step [18/415], Loss: 1.4539
Epoch [1/20], Step [19/415], Loss: 0.5003
Epoch [1/20], Step [20/415], Loss: 0.3885
Epoch [1/20], Step [21/415], Loss: 0.7826
Epoch [1/20], Step [22/415], Loss: 1.0995
Epoch [1/20], Step [23/415], Loss: 2.0780
Epoch [1/20], Step [24/415], Loss: 0.985

Epoch [1/20], Step [196/415], Loss: 0.5890
Epoch [1/20], Step [197/415], Loss: 0.3483
Epoch [1/20], Step [198/415], Loss: 0.7048
Epoch [1/20], Step [199/415], Loss: 0.2141
Epoch [1/20], Step [200/415], Loss: 0.3710
Epoch [1/20], Step [201/415], Loss: 0.4910
Epoch [1/20], Step [202/415], Loss: 1.4164
Epoch [1/20], Step [203/415], Loss: 0.2822
Epoch [1/20], Step [204/415], Loss: 1.1122
Epoch [1/20], Step [205/415], Loss: 0.5280
Epoch [1/20], Step [206/415], Loss: 0.3260
Epoch [1/20], Step [207/415], Loss: 0.3326
Epoch [1/20], Step [208/415], Loss: 0.4544
Epoch [1/20], Step [209/415], Loss: 1.1716
Epoch [1/20], Step [210/415], Loss: 0.4182
Epoch [1/20], Step [211/415], Loss: 0.6174
Epoch [1/20], Step [212/415], Loss: 0.9278
Epoch [1/20], Step [213/415], Loss: 0.6721
Epoch [1/20], Step [214/415], Loss: 1.4339
Epoch [1/20], Step [215/415], Loss: 0.4179
Epoch [1/20], Step [216/415], Loss: 0.7861
Epoch [1/20], Step [217/415], Loss: 0.5739
Epoch [1/20], Step [218/415], Loss: 0.4924
Epoch [1/20

Epoch [1/20], Step [388/415], Loss: 0.2473
Epoch [1/20], Step [389/415], Loss: 0.1188
Epoch [1/20], Step [390/415], Loss: 0.6738
Epoch [1/20], Step [391/415], Loss: 0.8716
Epoch [1/20], Step [392/415], Loss: 0.4946
Epoch [1/20], Step [393/415], Loss: 0.4610
Epoch [1/20], Step [394/415], Loss: 1.2655
Epoch [1/20], Step [395/415], Loss: 0.9676
Epoch [1/20], Step [396/415], Loss: 0.4148
Epoch [1/20], Step [397/415], Loss: 0.5456
Epoch [1/20], Step [398/415], Loss: 0.2069
Epoch [1/20], Step [399/415], Loss: 0.9014
Epoch [1/20], Step [400/415], Loss: 0.8365
Epoch [1/20], Step [401/415], Loss: 0.2443
Epoch [1/20], Step [402/415], Loss: 1.0581
Epoch [1/20], Step [403/415], Loss: 0.6320
Epoch [1/20], Step [404/415], Loss: 0.6671
Epoch [1/20], Step [405/415], Loss: 0.9018
Epoch [1/20], Step [406/415], Loss: 0.8602
Epoch [1/20], Step [407/415], Loss: 0.4844
Epoch [1/20], Step [408/415], Loss: 0.4906
Epoch [1/20], Step [409/415], Loss: 0.4836
Epoch [1/20], Step [410/415], Loss: 0.3313
Epoch [1/20

Epoch [2/20], Step [165/415], Loss: 0.3985
Epoch [2/20], Step [166/415], Loss: 0.3859
Epoch [2/20], Step [167/415], Loss: 0.3447
Epoch [2/20], Step [168/415], Loss: 0.2060
Epoch [2/20], Step [169/415], Loss: 0.1243
Epoch [2/20], Step [170/415], Loss: 0.7183
Epoch [2/20], Step [171/415], Loss: 0.4811
Epoch [2/20], Step [172/415], Loss: 0.1315
Epoch [2/20], Step [173/415], Loss: 0.6185
Epoch [2/20], Step [174/415], Loss: 0.4253
Epoch [2/20], Step [175/415], Loss: 0.4449
Epoch [2/20], Step [176/415], Loss: 0.3888
Epoch [2/20], Step [177/415], Loss: 0.1926
Epoch [2/20], Step [178/415], Loss: 0.2082
Epoch [2/20], Step [179/415], Loss: 0.5822
Epoch [2/20], Step [180/415], Loss: 0.7512
Epoch [2/20], Step [181/415], Loss: 0.3384
Epoch [2/20], Step [182/415], Loss: 0.3408
Epoch [2/20], Step [183/415], Loss: 0.1955
Epoch [2/20], Step [184/415], Loss: 0.4876
Epoch [2/20], Step [185/415], Loss: 0.4220
Epoch [2/20], Step [186/415], Loss: 0.2200
Epoch [2/20], Step [187/415], Loss: 0.5163
Epoch [2/20

...

Epoch [18/20], Step [400/415], Loss: 0.2351
Epoch [18/20], Step [401/415], Loss: 0.1396
Epoch [18/20], Step [402/415], Loss: 0.1515
Epoch [18/20], Step [403/415], Loss: 0.1196
Epoch [18/20], Step [404/415], Loss: 0.1350
Epoch [18/20], Step [405/415], Loss: 0.1156
Epoch [18/20], Step [406/415], Loss: 0.1883
Epoch [18/20], Step [407/415], Loss: 0.1365
Epoch [18/20], Step [408/415], Loss: 0.1979
Epoch [18/20], Step [409/415], Loss: 0.1566
Epoch [18/20], Step [410/415], Loss: 0.3698
Epoch [18/20], Step [411/415], Loss: 0.1716
Epoch [18/20], Step [412/415], Loss: 0.2822
Epoch [18/20], Step [413/415], Loss: 0.1336
Epoch [18/20], Step [414/415], Loss: 0.0816
Epoch [18/20], Step [415/415], Loss: 0.0770
Epoch [19/20], Step [1/415], Loss: 0.4213
Epoch [19/20], Step [2/415], Loss: 0.1149
Epoch [19/20], Step [3/415], Loss: 0.1181
Epoch [19/20], Step [4/415], Loss: 0.1037
Epoch [19/20], Step [5/415], Loss: 0.1234
Epoch [19/20], Step [6/415], Loss: 0.1407
Epoch [19/20], Step [7/415], Loss: 0.1591
Ep

Epoch [19/20], Step [175/415], Loss: 0.1472
Epoch [19/20], Step [176/415], Loss: 0.0859
Epoch [19/20], Step [177/415], Loss: 0.1244
Epoch [19/20], Step [178/415], Loss: 0.1214
Epoch [19/20], Step [179/415], Loss: 0.1812
Epoch [19/20], Step [180/415], Loss: 0.0501
Epoch [19/20], Step [181/415], Loss: 0.0447
Epoch [19/20], Step [182/415], Loss: 0.1528
Epoch [19/20], Step [183/415], Loss: 0.1551
Epoch [19/20], Step [184/415], Loss: 0.1149
Epoch [19/20], Step [185/415], Loss: 0.0664
Epoch [19/20], Step [186/415], Loss: 0.0654
Epoch [19/20], Step [187/415], Loss: 0.1734
Epoch [19/20], Step [188/415], Loss: 0.1373
Epoch [19/20], Step [189/415], Loss: 0.1117
Epoch [19/20], Step [190/415], Loss: 0.2252
Epoch [19/20], Step [191/415], Loss: 0.1001
Epoch [19/20], Step [192/415], Loss: 0.0518
Epoch [19/20], Step [193/415], Loss: 0.0815
Epoch [19/20], Step [194/415], Loss: 0.0780
Epoch [19/20], Step [195/415], Loss: 0.0418
Epoch [19/20], Step [196/415], Loss: 0.1304
Epoch [19/20], Step [197/415], L

Epoch [19/20], Step [364/415], Loss: 0.0578
Epoch [19/20], Step [365/415], Loss: 0.0445
Epoch [19/20], Step [366/415], Loss: 0.1191
Epoch [19/20], Step [367/415], Loss: 0.1146
Epoch [19/20], Step [368/415], Loss: 0.1678
Epoch [19/20], Step [369/415], Loss: 0.1450
Epoch [19/20], Step [370/415], Loss: 0.1014
Epoch [19/20], Step [371/415], Loss: 0.3695
Epoch [19/20], Step [372/415], Loss: 0.1843
Epoch [19/20], Step [373/415], Loss: 0.1230
Epoch [19/20], Step [374/415], Loss: 0.1993
Epoch [19/20], Step [375/415], Loss: 0.1029
Epoch [19/20], Step [376/415], Loss: 0.1167
Epoch [19/20], Step [377/415], Loss: 0.1160
Epoch [19/20], Step [378/415], Loss: 0.0805
Epoch [19/20], Step [379/415], Loss: 0.2259
Epoch [19/20], Step [380/415], Loss: 0.1201
Epoch [19/20], Step [381/415], Loss: 0.0862
Epoch [19/20], Step [382/415], Loss: 0.2368
Epoch [19/20], Step [383/415], Loss: 0.0824
Epoch [19/20], Step [384/415], Loss: 0.1709
Epoch [19/20], Step [385/415], Loss: 0.1878
Epoch [19/20], Step [386/415], L

Epoch [20/20], Step [139/415], Loss: 0.1126
Epoch [20/20], Step [140/415], Loss: 0.0746
Epoch [20/20], Step [141/415], Loss: 0.0601
Epoch [20/20], Step [142/415], Loss: 0.1124
Epoch [20/20], Step [143/415], Loss: 0.0459
Epoch [20/20], Step [144/415], Loss: 0.1012
Epoch [20/20], Step [145/415], Loss: 0.0698
Epoch [20/20], Step [146/415], Loss: 0.0184
Epoch [20/20], Step [147/415], Loss: 0.0784
Epoch [20/20], Step [148/415], Loss: 0.0361
Epoch [20/20], Step [149/415], Loss: 0.1247
Epoch [20/20], Step [150/415], Loss: 0.1315
Epoch [20/20], Step [151/415], Loss: 0.2103
Epoch [20/20], Step [152/415], Loss: 0.0995
Epoch [20/20], Step [153/415], Loss: 0.0478
Epoch [20/20], Step [154/415], Loss: 0.0748
Epoch [20/20], Step [155/415], Loss: 0.1438
Epoch [20/20], Step [156/415], Loss: 0.1114
Epoch [20/20], Step [157/415], Loss: 0.2907
Epoch [20/20], Step [158/415], Loss: 0.0872
Epoch [20/20], Step [159/415], Loss: 0.0368
Epoch [20/20], Step [160/415], Loss: 0.4995
Epoch [20/20], Step [161/415], L

Epoch [20/20], Step [328/415], Loss: 0.1845
Epoch [20/20], Step [329/415], Loss: 0.1057
Epoch [20/20], Step [330/415], Loss: 0.1705
Epoch [20/20], Step [331/415], Loss: 0.2365
Epoch [20/20], Step [332/415], Loss: 0.0682
Epoch [20/20], Step [333/415], Loss: 0.0588
Epoch [20/20], Step [334/415], Loss: 0.0598
Epoch [20/20], Step [335/415], Loss: 0.0737
Epoch [20/20], Step [336/415], Loss: 0.0700
Epoch [20/20], Step [337/415], Loss: 0.0884
Epoch [20/20], Step [338/415], Loss: 0.0237
Epoch [20/20], Step [339/415], Loss: 0.0682
Epoch [20/20], Step [340/415], Loss: 0.0716
Epoch [20/20], Step [341/415], Loss: 0.1074
Epoch [20/20], Step [342/415], Loss: 0.2152
Epoch [20/20], Step [343/415], Loss: 0.1704
Epoch [20/20], Step [344/415], Loss: 0.1757
Epoch [20/20], Step [345/415], Loss: 0.0810
Epoch [20/20], Step [346/415], Loss: 0.1634
Epoch [20/20], Step [347/415], Loss: 0.1943
Epoch [20/20], Step [348/415], Loss: 0.1732
Epoch [20/20], Step [349/415], Loss: 0.0503
Epoch [20/20], Step [350/415], L

In [25]:
best_epoch

3

In [26]:
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    for i in test_loader:
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

In [27]:
G,P = total_labels, total_preds

In [28]:
rmse(G,P)

1.1210935464541552

In [29]:
mse(G,P)

1.256850739901155

In [30]:
pearson(G,P)

0.07543580944650148

In [31]:
ci(G,P)

0.5410402981393629